In [1]:
from pathlib import Path
import sys
import os

sys.path.append((Path(os.getcwd()).parent / "src").as_posix())
sys.path.append((Path(os.getcwd())).parent.as_posix())

from analysis import VGG16Inspector, ResNet50Inspector
from datasets import CIFAR10, FMNIST, SPARK, JigsawCIFAR10
from datasets.base import DataLoader
from misc.trainer import Trainer
from models import PLAD

import torch
import matplotlib.pyplot as plt

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

/opt/conda/envs/perturbations/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CIFAR10
# best_model = "/data-isilon/results/perturbations/lightning_logs/train_features_cifar10/version_0/checkpoints/epoch=45-step=71898.ckpt"
# save_folder = "/data-isilon/datasets/perturbations/features/cifar10/train_features_cifar10/version_0/"

# SPARK
# best_model = "/data-isilon/results/perturbations/lightning_logs/train_features_spark/0000-0237-0466/checkpoints/epoch=16-step=39848.ckpt"
# save_folder = "/data-isilon/datasets/perturbations/features/spark/train_features_spark/0000-0237-0466/"

# CIFAR10 VGG16
# best_model = "/data-isilon/results/perturbations/lightning_logs/train_features_cifar10_vgg16/0007-0636-0236/checkpoints/start.ckpt"
# save_folder = "/data-isilon/datasets/perturbations/features/cifar10/train_features_cifar10_vgg16/0007-0636-0236/"
# inspector_class=VGG16Inspector

# CIFAR10 ResNet50
best_model = "/data-isilon/results/perturbations/lightning_logs/train_features_cifar10_resnet50/0000-0969-0933/checkpoints/start.ckpt"
save_folder = "/data-isilon/datasets/perturbations/features/cifar10/train_features_cifar10_resnet50/0000-0969-0933/"
inspector_class=ResNet50Inspector

# CIFAR10

In [7]:
trainer = Trainer(
    devices=[0],
    accelerator="gpu",
    logger=False,
)

class CIFAR10Bypass(CIFAR10):
    def train_dataloader(self):
        return DataLoader(
            self.train_set,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=False,
            shuffle=False,
        )

datamodule = CIFAR10Bypass(
    batch_size=16,
    dataset_dir="/data-isilon/datasets",
    resize=inspector_class.input_shape()
)

# datamodule.predict_dataloader = datamodule.train_dataloader
# save_file = "train"

datamodule.predict_dataloader = datamodule.val_dataloader
save_file = "val"

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# SPARK

In [31]:
from typing import Optional


trainer = Trainer(
    devices=[3],
    accelerator="gpu",
    logger=False,
)

class SPARKBypass(SPARK):
    def train_dataloader(self):
        return DataLoader(
            self.train_set,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            pin_memory=False,
            shuffle=False,
        )
    
    def setup(self, stage: str | None = None) -> None:
        kwargs = self._get_dataset_args()

        self.train_set = self.dataset_class(
            train=True,
            **kwargs,
        )

        self.val_set = self.dataset_class(
            train=False,
            **kwargs,
        )

    @property
    def num_classes(self) -> int:
        return self.dataset_class.num_classes()

datamodule = SPARKBypass(
    batch_size=64,
    dataset_dir="/data-isilon/datasets",
    resize=inspector_class.input_shape(),
)

datamodule.predict_dataloader = datamodule.train_dataloader
save_file = "train"

# datamodule.predict_dataloader = datamodule.val_dataloader
# save_file = "val"

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# Predict

In [4]:
model = inspector_class.load_from_checkpoint(best_model)

In [5]:
preds = trainer.predict(model, datamodule=datamodule)
preds = torch.cat(preds)

Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/envs/perturbations/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 625/625 [01:17<00:00,  8.02it/s]


In [6]:
preds.shape

torch.Size([10000, 3072])

In [7]:
torch.save(preds, f"{save_folder}/{save_file}.pt")